# Notes on running this notebook:
Copy the repository https://github.com/dennybritz/cnn-text-classification-tf one directory up from this one  
Switch to a virtual environment with tensorflow==1.9  
cd ../cnn-text-classification-tf
run ./train.py
Using the same environment you used to run the training script, you should be able to run this notebook.  
The output is a numpy array with the embedding of each sentence  
The inputs to the model are word-tokenized sentences

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import sys
sys.path.append('../cnn-text-classification-tf/')
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn
import csv
assert tf.__version__[0] == '1'

C:\Users\Aegisworn\Anaconda3\envs\style_detection\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aegisworn\Anaconda3\envs\style_detection\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aegisworn\Anaconda3\envs\style_detection\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aegisworn\Anaconda3\en

In [13]:
batch_size = 64
checkpoint_dir = '../cnn-text-classification-tf/runs/1604974730/checkpoints/'
positive_data_file = '../cnn-text-classification-tf/data/rt-polaritydata/rt-polarity.pos'
negative_data_file = '../cnn-text-classification-tf/data/rt-polaritydata/rt-polarity.neg'
allow_soft_placement = True
log_device_placement = False

In [8]:
x_raw, y_test = data_helpers.load_data_and_labels(positive_data_file, negative_data_file)
y_test = np.argmax(y_test, axis=1)

vocab_path = os.path.join(checkpoint_dir, '..', 'vocab')
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

Instructions for updating:
Please use tensorflow/transform or tf.data.


In [32]:
# example input sentences
print(x_raw[:10])

["the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal", "the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words cannot adequately describe co writer director peter jackson 's expanded vision of j r r tolkien 's middle earth", 'effective but too tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start', "emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one", 'the film provides some great insight into the neurotic mindset of all comics even those who have reached the absolute top of the game', 'offers that rare combination of entertainment and education', 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions', "steers turns in a snappy screenplay that curls at t

In [29]:
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        embeddings = graph.get_operation_by_name('dropout/dropout/mul').outputs[0]
        
        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), batch_size, 1, shuffle=False)

        # Collect the predictions here

        for i, x_test_batch in enumerate(batches):
            batch_embeddings = sess.run(embeddings, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            if i == 0:
                all_embeddings = batch_embeddings
            else:
                all_embeddings = np.concatenate([all_embeddings, batch_embeddings], axis=0)

INFO:tensorflow:Restoring parameters from C:\Users\Aegisworn\code\deep_fakes\Project\cnn-text-classification-tf\runs\1604974730\checkpoints\model-30000


In [31]:
all_embeddings.shape

(10662, 384)